In [13]:
import thermofun as fun #thermo. properties at T-P-X
import reaktoro as rkt # Gibbs free energy minimization

In [14]:
# create base dataset from mines19 data file
database = fun.Database("../data/raw/thermodynamic_data/mines19-thermofun.json");

# add critical parameters for gases (missing in ThermoFun data files)
database.appendData("../data/raw/thermodynamic_data/slop98-gases-crit-thermofun.json");

# add Co aqueous species
database.appendData("../data/raw/thermodynamic_data/Co_aq_species_thermo.json");
database.appendData("../data/raw/thermodynamic_data/Co-S_aq_species_thermo_Migdisov_etal_2011_OptimC.json");
# database.appendData("../data/raw/thermodynamic_data/Co-Cl_aq_species_thermo_Liu_etal_2011.json");
database.appendData("../data/raw/thermodynamic_data/Co-Cl_aq_species_thermo_Migdisov_etal_2011_OptimC.json");

# add Co minerals 
database.appendData("../data/raw/thermodynamic_data/Co_mineral_thermo.json");

db = rkt.ThermoFunDatabase(database)

The element with symbol H is already in the database. Overwritting ...
To add it to the database as a separate record assign it a different symbol.
The element with symbol C is already in the database. Overwritting ...
To add it to the database as a separate record assign it a different symbol.
The element with symbol O is already in the database. Overwritting ...
To add it to the database as a separate record assign it a different symbol.
The element with symbol S is already in the database. Overwritting ...
To add it to the database as a separate record assign it a different symbol.
The substance with symbol H2O(g) is already in the database. Overwritting ...
To add it to the database as a separate record assign it a different symbol.
The substance with symbol CH4(g) is already in the database. Overwritting ...
To add it to the database as a separate record assign it a different symbol.
The substance with symbol CO2(g) is already in the database. Overwritting ...
To add it to the dat

In [31]:
aqueous_phase = rkt.AqueousPhase(rkt.speciate("H O Na Cl Co S"))
mineral_phases = rkt.MineralPhases(["Linnaeite", "Cattierite", "Co-pentlandite", "CoO(s)", "Co3O4(s)", "Co(s)"])

phases = rkt.Phases(db)

phases.add(aqueous_phase)
phases.add(mineral_phases)

system = rkt.ChemicalSystem(phases)


In [33]:
print([species.name() for species in system.species()])

['Cl-', 'ClO-', 'ClO4-', 'Co+2', 'Co+3', 'CoCl+', 'CoCl2@', 'CoCl3-', 'CoCl4-2', 'CoH2S+2', 'CoHS+', 'CoO2-2', 'CoO@', 'CoOH+', 'CoOH+2', 'H+', 'H2@', 'H2O2@', 'H2O@', 'H2S2O3@', 'H2S2O4@', 'H2S@', 'HCl@', 'HClO@', 'HCoO2-', 'HO2-', 'HS-', 'HS2O3-', 'HS2O4-', 'HSO3-', 'HSO4-', 'HSO5-', 'Na(SO4)-', 'Na+', 'NaCl@', 'NaOH@', 'O2@', 'OH-', 'S2-2', 'S2O3-2', 'S2O4-2', 'S2O5-2', 'S2O6-2', 'S2O8-2', 'S3-2', 'S3O6-2', 'S4-2', 'S4O6-2', 'S5-2', 'S5O6-2', 'SO2@', 'SO3-2', 'SO4-2', 'Linnaeite', 'Cattierite', 'Co-pentlandite', 'CoO(s)', 'Co3O4(s)', 'Co(s)']


# Reactions

## mineral-mineral 
### linnaeite-Cobaltpentlandite 
$$3Co_3S_4 + 4H_2O° \leftrightarrow Co_9S_8 + 4H_2S° + 2O_2°$$ 
$$3Co_3S_4 + 4H_2O° \leftrightarrow Co_9S_8 + 4HS^- + 4H^+ + 2O_2°$$ 
$$3Co_3S_4 + 4H^+ + 8O_2° \leftrightarrow Co_9S_8 + 4HSO_4^-$$ 
$$3Co_3S_4 + 4H_2O° + 6O_2° \leftrightarrow Co_9S_8 + 4SO_4^{-2} + 8H^+$$ 

### cattierite-linnaeite
$$3CoS_2 + 2H_2O° \leftrightarrow Co_3S_4 + 2H_2S° + O_2°$$
$$3CoS_2 +  2H_2O° \leftrightarrow Co_3S_4 + 2HS^- + 2H^+ + O_2°$$
$$3CoS_2 + 2H_2O° + 3O_2° \leftrightarrow Co_3S_4 + 2HSO_4^- + 2H^+$$
$$3CoS_2 + 2H_2O° + 6O_2° \leftrightarrow Co_3S_4 + 2SO_4^{-2} + 4H^+$$

### $Co_3O_4$-Cobaltpentlandite
$$3Co_3O_4 + 8HSO_4^- + 8H^+ \leftrightarrow Co_9S_8 + 8H_2O° + 12O_2°$$
$$3Co_3O_4 + 8SO_4^{-2} + 16H^+ \leftrightarrow Co_9S_8 + 8H_2O° + 12O_2°$$

## mineral-aq species

## aq species-aq species (predominance)



In [53]:
min_min_rxns = ["3Linnaeite + 4H2O@         = Co-pentlandite + 4H2S@ + 2O2@", 
                "3Linnaeite + 4H2O@         = Co-pentlandite + 4HS- + 4H+ + 2O2@", 
                "3Linnaeite + 4H2O@ + 6O2@  = Co-pentlandite + 4SO4-2 + 8H+", 
                "3Cattierite + 2H2O@        = Linnaeite + 2H2S@ + O2@", 
                "3Cattierite + 2H2O@        = Linnaeite + 2HS- + 2H+ + O2@",
                "3Cattierite + 2H2O@ + 3O2@ = Linnaeite + 2HSO4- + 2H+", 
                "3Cattierite + 2H2O@ + 3O2@ = Linnaeite + 2SO4-2 + 4H+", 
                "3Co3O4(s) + 4HSO4- + 4H+   = Linnaeite + 4H2O@ + 8O2@",
                "3Co3O4(s) + 8SO4-2 + 16H+  = Co-pentlandite + 8H2O@ + 18O2@"
                ]
aq_aq_rxns = ["H2S@        = HS- + H+", 
              "H2S@ + 2O2@ = HSO4- + H+", 
              "H2S@ + 2O2@ = SO4-2 + 2H+", 
              "HS- + 2O2@  = SO4-2 + H+", 
              "HSO4-       = SO4-2 + H+"
             ]

min_aq_rxns = ["Co-pentlandite + 36Cl- + 18H+ + 0.5O2@ = 9CoCl4-2 + 8H2S@ + H2O@", 
               "Linnaeite + 12Cl- + 6H+ + H2O@         = 3CoCl4-2 + 4H2S@ + 0.5O2@", 
               "Cattierite + 4Cl- + 2H+ + H2O@         = CoCl4-2 + 2H2S@ + 0.5O2@", 
               "Cattierite + 4Cl- + H2O@ + 3.5O2@      = CoCl4-2 + 2HSO4-", 
               "Cattierite + 4Cl- + H2O@ + 3.5O2@      = CoCl4-2 + 2SO4-2 + 2H+", 
               "Linnaeite + 12Cl- + H2O@ + 7.5O2@      = 3CoCl4-2 + 4SO4-2 + 2H+", 
               "Co-pentlandite + 36Cl- + 2H+ + 16.5O2@ = 9CoCl4-2 + 8SO4-2 + H2O@", 
               "Co3O4(s) + 12Cl- + 6H+                 = 3CoCl4-2 + 3H2O@ + 0.5O2@"
              ]

In [35]:
engine = fun.ThermoEngine(database)

In [56]:
T = 150+273.15
P = rkt.waterSaturatedPressureWagnerPruss(T)[0]/100000 # [bar] 

-6.497702402658512
88.95010784448733
85.22446770261519
91.72217010527369
-3.725640141872137


In [57]:
min_min_logK = []
for rxn in min_min_rxns: 
    rxn_properties = engine.thermoPropertiesReaction(T, P, rxn) 
    min_min_logK.append(rxn_properties.log_equilibrium_constant.val)

min_aq_logK = []
for rxn in min_aq_rxns: 
    rxn_properties = engine.thermoPropertiesReaction(T, P, rxn) 
    min_aq_logK.append(rxn_properties.log_equilibrium_constant.val)

aq_aq_logK = []
for rxn in aq_aq_rxns: 
    rxn_properties = engine.thermoPropertiesReaction(T, P, rxn) 
    aq_aq_logK.append(rxn_properties.log_equilibrium_constant.val)


In [ ]:
# vertical 

# horizontal 

# both 